In [ ]:
import numpy as np
import pandas as pd
import gc
from copy import copy
from tqdm import trange
from datetime import datetime
import warnings
from sklearn.metrics.pairwise import pairwise_kernels
from AssetPricing import factor_model, backtest, kernel_methods
from AssetPricing.kernel_methods import AlwaysNoneList, kernel_ic_maximizer
import os
from joblib import Parallel, delayed
import numpy as np
import pickle
import os
warnings.filterwarnings('ignore', message = 'Unused variable')

In [ ]:
class AlwaysNoneList:
    def __getitem__(self, index):
        return None


In [ ]:
data = pd.read_csv("data/cha1.csv", index_col=0)
# data = pd.read_csv('D:/project/data/data.csv', index_col = 1)
date = list(pd.unique(data.index))
date.sort()
ret = data.pivot(columns = 'PERMNO', values = 'RET').fillna(0).loc[date]

characteristics = dict()
for t in trange(len(date)):
    key = date[t]
    cha = data.loc[key].set_index('PERMNO').iloc[:, 1:]    
    cha = 2*cha.rank()/cha.count()-1
    rt = ret.loc[key, cha.index]
    cha.loc[rt[rt == 0].index] = np.nan
    cha = cha.dropna(how = 'all')
    
    characteristics[key] = cha

ret[ret == 0] = np.nan
ret = ret.add(- ret.mean(axis = 1), axis = 0).divide(ret.std(axis = 1), axis = 0)
ret = ret.fillna(0)

In [ ]:
file_path = "data/saved_data/dictionary for prediction dictionary.pkl"

with open(file_path, "rb") as file:

    dict_of_dict = pickle.load(file)

cov_mat_2_optim = kernel_methods.cov_mat(dict_of_dict)

model = kernel_ic_maximizer(characteristics, ret)

model.get_return_dict()

cor_ls_2_optim = kernel_methods.cor_ls(dict_of_dict, model.return_dict)

numerator = np.linalg.inv(cov_mat_2_optim).dot(cor_ls_2_optim)

denominator = np.ones((1, cor_ls_2_optim.shape[0])).dot(np.linalg.inv(cov_mat_2_optim).dot(cor_ls_2_optim))

optimal_weight = numerator/denominator

In [ ]:
file_path = "data/saved_data/dictionary for prediction dictionary.pkl"

with open(file_path, "rb") as file:

    dict_of_dict = pickle.load(file)

cov_mat_2_optim = kernel_methods.cov_mat(dict_of_dict)

model = kernel_ic_maximizer(characteristics, ret)

model.get_return_dict()

cov_ls_2_optim = kernel_methods.cov_ls(dict_of_dict, model.return_dict)

negative_mask = cov_ls_2_optim < 0

cov_ls_2_optim = np.abs(cov_ls_2_optim)

numerator = np.linalg.inv(cov_mat_2_optim).dot(cov_ls_2_optim)

denominator = np.ones((1, cov_ls_2_optim.shape[0])).dot(np.linalg.inv(cov_mat_2_optim).dot(cov_ls_2_optim))

optimal_weight = numerator/denominator

optimal_weight[negative_mask] = -optimal_weight[negative_mask]

In [ ]:
optimal_weight

In [ ]:
list(optimal_weight.squeeze())

In [5]:
rolling_lag_dict = model.get_prediction_of_rolling_lag(lag_ls=list(range(1,13)), weight_ls = list(optimal_weight.squeeze()), lamb = 1e-3, metric = "linear",
                                      gamma = 1e-6, degree = None, coef0 = None)

100%|██████████| 611/611 [54:34<00:00,  5.36s/it]  


In [6]:
kernel_methods.IC_of_two_dict(rolling_lag_dict, model.return_dict)

0.07582951730085213


0.07582951730085213

In [ ]:
"""
n_jobs, lag_ls = AlwaysNoneList(), lamb = 1e-3, metric = "linear", 
                      gamma_ls = AlwaysNoneList(), degree_ls = AlwaysNoneList(), coef_ls = AlwaysNoneList()

"""
parameters = {
    "n_jobs" : 7,
    "lag_ls" : [1,2,3,4,5,6,7,8,9,10,11,12],
    "lamb" :1e-3,
    "metric": "rbf",
    "gamma_ls": [1e-6, 1e-6, 1e-6,1e-6, 1e-6, 1e-6,1e-6, 1e-6, 1e-6,1e-6, 1e-6, 1e-6],
}

model = kernel_ic_maximizer(characteristics, ret)

model.cal_pred_dict(**parameters)

In [ ]:
parameters = {
    "characteristics" : characteristics,
    "ret" : ret,
    "n_jobs" :7,
    "kernel_method": "rbf",
    "time_lags": range(1,7),
    "gamma_range": np.logspace(0,-6,2),
}

results_dict = kernel_methods.run_parallel_cal_IC(**parameters)



